In [4]:
import json
import numpy as np
import time
from bson.objectid import ObjectId
from gridfs import GridFS
from pymongo import MongoClient
from PIL import Image
from padma.models.encoder.encoder import EncoderModel, SIZE
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import generate_batch, get_imagens_recortadas

db = MongoClient(host=MONGODB_URI)[DATABASE]
encoder = EncoderModel()

In [5]:
batch_size = 10
limit = 100
TEMBBOX = {'metadata.contentType': 'image/jpeg',
          'metadata.predictions.bbox': {'$exists': True}}
dict_filtro = TEMBBOX
projection = ['metadata.predictions']
dict_filtro['metadata.predictions.index'] = {'$exists': False}
batch_gen = generate_batch(db, filtro=dict_filtro, projection=projection,
                           batch_size=batch_size, limit=limit)

## Abaixo, faz predição e já grava, registro a registro, no Banco de Dados

In [6]:
X = np.zeros((batch_size, *SIZE, 1), dtype=np.float32)
rows = []
y = []
s = time.time()
for batch, rows in batch_gen:
    print(rows)
    if len(batch) == 0:
        break
    for i, (images, row) in enumerate(zip(batch, rows)):
        image_array = encoder.image_prepare(images[0])
        s0 = time.time()
        X[i, :, :, :] = image_array
        rows.append(row)
    s1 = time.time()
    print('Montou X em %0.2f ' % (s1 - s0))
    indexes = encoder.model.predict(X)
    indexes = indexes.reshape(-1, 128).astype(np.float32)
    y.append(indexes)
    s2 = time.time()
    print('Fez predição em %s' % (s2 - s1))
    # print(indexes)
    ystack = np.vstack(y).astype(np.float32)
    for i in range(batch_size):
        new_list = ystack[i, :].tolist()
        index_row = rows[i]
        _id = index_row['_id']
        old_predictions = index_row['metadata']['predictions']
        # print(old_predictions)
        new_predictions = old_predictions
        new_predictions[0]['index'] = new_list
        # print(new_predictions)
        result = db.fs.files.update_one({'_id': ObjectId(_id)}, {'$set': {'metadata.predictions': json.dumps(new_predictions)}})
        # print(result.matched_count, result.modified_count)
        print(_id)
    s3 = time.time()
    print('Atualizou Banco de Dados em %s' % (s3 - s2))
s4 = time.time()
elapsed = s4 - s
print('Tempo total: %s. Por imagem: %s' % (elapsed, elapsed / limit))


[]
Tempo total: 0.0979607105255127. Por imagem: 0.000979607105255127
